In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from scipy import sparse
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nltk
from nltk.corpus import stopwords
import string
# Any results you write to the current directory are saved as output.
stop_words = set(stopwords.words('english') + list(string.punctuation))

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
tr_df = pd.read_csv("../input/train.csv")
te_df = pd.read_csv("../input/test.csv")

In [3]:
print("Train size: ", tr_df.shape[0])
print("Test size: ", te_df.shape[0])

Train size:  16000
Test size:  4491


In [4]:
tr_df["label"].value_counts()

5    6891
4    4774
3    1716
2    1440
1    1179
Name: label, dtype: int64

In [5]:
te_df["label"].value_counts()

-1    4491
Name: label, dtype: int64

In [6]:
def tokenize(text):
    '''
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    https://textminingonline.com/dive-into-nltk-part-ii-sentence-tokenize-and-word-tokenize
    e.g. 
    Input: 'It is a nice day. I am happy.'
    Output: ['it', 'is', 'a', 'nice', 'day', 'i', 'am', 'happy']
    '''
    tokens = []
    # YOUR CODE HERE
    for word in nltk.word_tokenize(text):
        word = word.lower()
        if word not in stop_words and not word.isnumeric():
            tokens.append(word)

    return tokens



In [7]:
#split train and val
N = len(tr_df['id'])
train_size = int(round(N * 0.8))
train_df = tr_df[:train_size]
val_df = tr_df[train_size:-1]
test_df = te_df


train_id = train_df['id']
val_id = val_df['id']
test_id = test_df['id']

train_label = train_df['label']
val_label = val_df['label']
test_label = test_df['label']



In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stop_words)

train_bow = vectorizer.fit_transform(train_df['text'])
val_bow = vectorizer.transform(val_df['text'])
test_bow = vectorizer.transform(test_df['text'])



In [9]:
from sklearn.feature_extraction.text import TfidfTransformer


transformer = TfidfTransformer()


train_tfidf = transformer.fit_transform(train_bow)
val_tfidf = transformer.transform(val_bow)
test_tfidf = transformer.transform(test_bow)

In [10]:
from sklearn import preprocessing 

train_tfidf = preprocessing.scale(train_tfidf, with_mean=False)

# train_bow = preprocessing.normalize(train_bow, axis=0)

In [11]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV



/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [12]:



tuned_parameters = [{'kernel': ['linear'], 'gamma': [1e-2, 1e-3],
                     'C': [5e-3, 1e-2,3e-2]}]

clf = GridSearchCV(SVC(), tuned_parameters, cv=3,scoring="accuracy", verbose=100)

clf.fit(train_bow, train_label)




Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.005, gamma=0.01, kernel=linear ..............................
[CV] ..... C=0.005, gamma=0.01, kernel=linear, score=0.583978 - 1.3min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s
[CV] C=0.005, gamma=0.01, kernel=linear ..............................
[CV] ..... C=0.005, gamma=0.01, kernel=linear, score=0.572433 - 1.3min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.6min remaining:    0.0s
[CV] C=0.005, gamma=0.01, kernel=linear ..............................
[CV] ..... C=0.005, gamma=0.01, kernel=linear, score=0.567878 - 1.3min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.0min remaining:    0.0s
[CV] C=0.005, gamma=0.001, kernel=linear .............................
[CV] .... C=0.005, gamma=0.001, kernel=linear, score=0.583978 - 1.3min
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  5.3min remaining:    0.0s
[CV] C=0.005, gamma=0.001, kernel=linear ...........

GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['linear'], 'gamma': [0.01, 0.001], 'C': [0.005, 0.01, 0.03]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy',
       verbose=100)

In [13]:
print("Best parameters")
print(clf.best_params_)
print()
print(clf.best_score_)

Best parameters
{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}

0.5834375


In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [16]:
mnb_clf = MultinomialNB()

mnb_clf.fit(train_bow, train_label)
print("train_acc: ")
print(mnb_clf.score(train_bow, train_label))
print("val_acc: ")
print(mnb_clf.score(val_bow, val_label))


train_acc: 
0.7803125
val_acc: 
0.6076899030947172


In [17]:
gnb_clf = GaussianNB()

gnb_clf.fit(train_bow.toarray(), train_label)
print("train_acc: ")
print(gnb_clf.score(train_bow.toarray(), train_label))
print("val_acc: ")
print(gnb_clf.score(val_bow.toarray(), val_label))

train_acc: 
0.79125
val_acc: 
0.3150984682713348


In [ ]:
from sklearn import tree
dt_clf = tree.DecisionTreeClassifier()
dt_clf = dt_clf.fit(train_tfidf.toarray(), train_label)

print("train_acc: ")
print(dt_clf.score(train_tfidf, train_label))
print("val_acc: ")
print(dt_clf.score(val_tfidf, val_label))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf = rf_clf.fit(train_tfidf, train_label)

print("train_acc: ")
print(rf_clf.score(train_tfidf, train_label))
print("val_acc: ")
print(rf_clf.score(val_tfidf, val_label))